In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Thiết lập ---

# Đường dẫn tệp từ các notebook của bạn
hotel_train_path = "/kaggle/input/uit-hotel-absa/Train.csv"
# Lưu ý: Sửa lại lỗi chính tả "resturant" trong đường dẫn gốc từ notebook của bạn
restaurant_train_path = "/kaggle/input/uit-absa-resturant-dataset/Train.csv"

# Ánh xạ nhãn từ Slide 4
label_map = {
    0: 'None (0)',
    1: 'Positive (1)',
    2: 'Negative (2)',
    3: 'Neutral (3)'
}

# Thư mục để lưu hình ảnh
output_dir = "visualizations"
os.makedirs(output_dir, exist_ok=True)

def plot_sentiment_distribution(csv_path, title, output_filename):
    """
    Hàm để tải, xử lý và vẽ biểu đồ phân phối nhãn cho một tệp CSV.
    """
    print(f"\nĐang xử lý: {title}")
    print(f"Kiểm tra tệp tại: {csv_path}")
    
    if not os.path.exists(csv_path):
        print(f"--- LỖI: Không tìm thấy tệp tại đường dẫn: {csv_path} ---")
        print("Điều này là do tệp CSV không tồn tại trong môi trường hiện tại.")
        print("Bạn cần chạy code này trong một môi trường (như Kaggle/Colab) nơi có các tệp dữ liệu đó,")
        print("hoặc tải các tệp CSV lên và cập nhật lại đường dẫn trong code.")
        print("--------------------------------------------------")
        return

    try:
        # Tải dữ liệu
        df = pd.read_csv(csv_path)
        
        # Xác định các cột khía cạnh (loại trừ cột 'Review')
        aspect_cols = [col for col in df.columns if col != 'Review']
        print(f"Tìm thấy {len(aspect_cols)} cột khía cạnh.")
        
        # Chuyển đổi dataframe sang dạng "dài" (long format)
        df_melted = df.melt(id_vars=['Review'], 
                            value_vars=aspect_cols, 
                            var_name='Aspect', 
                            value_name='Sentiment')
        
        # Ánh xạ nhãn số sang nhãn văn bản
        df_melted['Sentiment_Label'] = df_melted['Sentiment'].map(label_map)
        
        # Sắp xếp thứ tự cho biểu đồ
        order = [label_map[0], label_map[1], label_map[2], label_map[3]]
        
        # Vẽ biểu đồ
        plt.figure(figsize=(10, 6))
        ax = sns.countplot(
            data=df_melted,
            x='Sentiment_Label',
            order=order,
            palette='viridis'
        )
        
        ax.set_title(f'Phân phối nhãn cảm xúc - {title}', fontsize=16)
        ax.set_xlabel('Loại cảm xúc', fontsize=12)
        ax.set_ylabel('Tổng số lượng (trên tất cả các khía cạnh)', fontsize=12)
        
        # Hiển thị số lượng trên mỗi cột
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha='center', va='center', 
                        xytext=(0, 9), 
                        textcoords='offset points')
        
        full_output_path = os.path.join(output_dir, output_filename)
        plt.tight_layout()
        plt.savefig(full_output_path)
        print(f"Đã lưu biểu đồ thành công vào: {full_output_path}")
        plt.close()

    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý {csv_path}: {e}")

# --- Chạy thực thi ---
print("Bắt đầu trực quan hóa bộ dữ liệu...")

# 1. Vẽ biểu đồ cho bộ Hotel
plot_sentiment_distribution(
    csv_path=hotel_train_path,
    title="UIT-Hotel-ABSA (Train Set)",
    output_filename="hotel_sentiment_distribution.png"
)

# 2. Vẽ biểu đồ cho bộ Restaurant
plot_sentiment_distribution(
    csv_path=restaurant_train_path,
    title="UIT-Restaurant-ABSA (Train Set)",
    output_filename="restaurant_sentiment_distribution.png"
)

print("\nHoàn tất quá trình.")

Bắt đầu trực quan hóa bộ dữ liệu...

Đang xử lý: UIT-Hotel-ABSA (Train Set)
Kiểm tra tệp tại: /kaggle/input/uit-hotel-absa/Train.csv
Tìm thấy 34 cột khía cạnh.
Đã lưu biểu đồ thành công vào: visualizations/hotel_sentiment_distribution.png

Đang xử lý: UIT-Restaurant-ABSA (Train Set)
Kiểm tra tệp tại: /kaggle/input/uit-absa-resturant-dataset/Train.csv
Tìm thấy 12 cột khía cạnh.
Đã lưu biểu đồ thành công vào: visualizations/restaurant_sentiment_distribution.png

Hoàn tất quá trình.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# --- Thiết lập ---

# Đường dẫn tệp từ các notebook của bạn
hotel_train_path = "/kaggle/input/uit-hotel-absa/Train.csv"
restaurant_train_path = "/kaggle/input/uit-absa-resturant-dataset/Train.csv"

# Ánh xạ nhãn từ Slide 4
label_map = {
    0: 'None (0)',
    1: 'Positive (1)',
    2: 'Negative (2)',
    3: 'Neutral (3)'
}
# Màu sắc tương ứng
colors = ['#d3d3d3', '#4CAF50', '#F44336', '#FFC107'] 
# (Xám cho None, Xanh cho Positive, Đỏ cho Negative, Vàng cho Neutral)

# Thư mục để lưu hình ảnh
output_dir = "visualizations_per_aspect"
os.makedirs(output_dir, exist_ok=True)

def plot_support_per_aspect(csv_path, title, output_filename, label_map, colors):
    """
    Hàm để tải, xử lý và vẽ biểu đồ support (số lượng) cho từng aspect.
    """
    print(f"\nĐang xử lý: {title}")
    print(f"Kiểm tra tệp tại: {csv_path}")
    
    if not os.path.exists(csv_path):
        print(f"--- LỖI: Không tìm thấy tệp tại đường dẫn: {csv_path} ---")
        print("Bỏ qua việc tạo biểu đồ này.")
        print("--------------------------------------------------")
        return

    try:
        # Tải dữ liệu
        df = pd.read_csv(csv_path)
        
        # Xác định các cột khía cạnh
        aspect_cols = [col for col in df.columns if col != 'Review']
        print(f"Tìm thấy {len(aspect_cols)} cột khía cạnh.")
        
        # Đếm support cho từng aspect
        # Áp dụng value_counts cho mỗi cột aspect
        support_df = df[aspect_cols].apply(pd.Series.value_counts).fillna(0).astype(int)
        
        # Đảm bảo tất cả các cột (0, 1, 2, 3) đều tồn tại, ngay cả khi support = 0
        support_df = support_df.reindex([0, 1, 2, 3], fill_value=0)
        
        # Chuyển vị (transpose) để các khía cạnh (aspects) là hàng
        support_df = support_df.transpose()
        
        # Đổi tên cột từ số (0, 1, 2, 3) sang nhãn văn bản
        support_df = support_df.rename(columns=label_map)
        
        # Sắp xếp các khía cạnh theo thứ tự bảng chữ cái để dễ nhìn
        support_df = support_df.sort_index(ascending=False)
        
        # --- Vẽ biểu đồ ---
        # Tính toán chiều cao động cho biểu đồ dựa trên số lượng khía cạnh
        # (Ví dụ: 0.4 inch mỗi khía cạnh + 3 inch lề)
        num_aspects = len(aspect_cols)
        fig_height = max(6, num_aspects * 0.4 + 3)
        
        ax = support_df.plot(
            kind='barh', 
            stacked=True, 
            color=colors,
            figsize=(14, fig_height),
            width=0.8
        )
        
        ax.set_title(f'Support (Số lượng mẫu) theo từng Khía cạnh - {title}', fontsize=18, pad=20)
        ax.set_xlabel('Số lượng mẫu (Support)', fontsize=14)
        ax.set_ylabel('Khía cạnh (Aspect)', fontsize=14)
        
        # Di chuyển legend ra ngoài biểu đồ
        ax.legend(
            title='Cảm xúc (Sentiment)',
            bbox_to_anchor=(1.02, 1), 
            loc='upper left', 
            fontsize='medium'
        )
        
        # Hiển thị lưới
        ax.grid(axis='x', linestyle='--', alpha=0.7)
        ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ','))) # Định dạng số
        
        plt.margins(y=0.01) # Giảm khoảng trắng ở trên và dưới
        
        full_output_path = os.path.join(output_dir, output_filename)
        plt.tight_layout(rect=[0, 0, 0.85, 1]) # Điều chỉnh layout để chừa chỗ cho legend
        plt.savefig(full_output_path)
        print(f"Đã lưu biểu đồ thành công vào: {full_output_path}")
        plt.close()

    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý {csv_path}: {e}")

# --- Chạy thực thi ---
print("Bắt đầu trực quan hóa support theo từng khía cạnh...")

# 1. Vẽ biểu đồ cho bộ Hotel
plot_support_per_aspect(
    csv_path=hotel_train_path,
    title="UIT-Hotel-ABSA (Train Set)",
    output_filename="hotel_support_per_aspect.png",
    label_map=label_map,
    colors=colors
)

# 2. Vẽ biểu đồ cho bộ Restaurant
plot_support_per_aspect(
    csv_path=restaurant_train_path,
    title="UIT-Restaurant-ABSA (Train Set)",
    output_filename="restaurant_support_per_aspect.png",
    label_map=label_map,
    colors=colors
)

print("\nHoàn tất quá trình.")

Bắt đầu trực quan hóa support theo từng khía cạnh...

Đang xử lý: UIT-Hotel-ABSA (Train Set)
Kiểm tra tệp tại: /kaggle/input/uit-hotel-absa/Train.csv
Tìm thấy 34 cột khía cạnh.
Đã lưu biểu đồ thành công vào: visualizations_per_aspect/hotel_support_per_aspect.png

Đang xử lý: UIT-Restaurant-ABSA (Train Set)
Kiểm tra tệp tại: /kaggle/input/uit-absa-resturant-dataset/Train.csv
Tìm thấy 12 cột khía cạnh.
Đã lưu biểu đồ thành công vào: visualizations_per_aspect/restaurant_support_per_aspect.png

Hoàn tất quá trình.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# --- Thiết lập ---

# Đường dẫn tệp từ các notebook của bạn
# !! Quan trọng: Đảm bảo các đường dẫn này chính xác !!
hotel_train_path = "/kaggle/input/uit-hotel-absa/Train.csv"
restaurant_train_path = "/kaggle/input/uit-absa-resturant-dataset/Train.csv"

# --- THAY ĐỔI: Chỉ bao gồm các nhãn 1, 2, 3 ---
label_map_no_none = {
    1: 'Positive (1)',
    2: 'Negative (2)',
    3: 'Neutral (3)'
}
# --- THAY ĐỔI: Loại bỏ màu xám của 'None' ---
colors_no_none = ['#4CAF50', '#F44336', '#FFC107'] 
# (Xanh cho Positive, Đỏ cho Negative, Vàng cho Neutral)

# Thư mục để lưu hình ảnh (ví dụ: /kaggle/working/visualizations_active)
output_dir = "visualizations_active_sentiments"
os.makedirs(output_dir, exist_ok=True)
print(f"Sẽ lưu biểu đồ vào thư mục: {os.path.abspath(output_dir)}")

def plot_active_support_per_aspect(csv_path, title, output_filename, label_map, colors):
    """
    Hàm để tải, xử lý và vẽ biểu đồ support (số lượng)
    CHỈ cho các nhãn khác 0 (None).
    """
    print(f"\nĐang xử lý: {title}")
    print(f"Kiểm tra tệp tại: {csv_path}")
    
    # Kiểm tra xem tệp có tồn tại không
    if not os.path.exists(csv_path):
        print(f"--- CẢNH BÁO: Không tìm thấy tệp tại đường dẫn: {csv_path} ---")
        print("Bỏ qua việc tạo biểu đồ này.")
        print("--------------------------------------------------")
        return

    try:
        # Tải dữ liệu
        df = pd.read_csv(csv_path)
        
        # Xác định các cột khía cạnh
        aspect_cols = [col for col in df.columns if col != 'Review']
        print(f"Tìm thấy {len(aspect_cols)} cột khía cạnh.")
        
        # Đếm support cho từng aspect
        support_df = df[aspect_cols].apply(pd.Series.value_counts).fillna(0).astype(int)
        
        # Đảm bảo các cột (0, 1, 2, 3) đều tồn tại
        support_df = support_df.reindex([0, 1, 2, 3], fill_value=0)
        
        # --- THAY ĐỔI QUAN TRỌNG: Loại bỏ hàng 'None (0)' ---
        if 0 in support_df.index:
            support_df = support_df.drop(0)
        # ----------------------------------------------------
        
        # Chuyển vị (transpose) để các khía cạnh (aspects) là hàng
        support_df = support_df.transpose()
        
        # Đổi tên cột từ số (1, 2, 3) sang nhãn văn bản
        support_df = support_df.rename(columns=label_map)
        
        # Sắp xếp các khía cạnh theo thứ tự bảng chữ cái (từ dưới lên trên)
        support_df = support_df.sort_index(ascending=False)
        
        # --- Vẽ biểu đồ ---
        num_aspects = len(aspect_cols)
        fig_height = max(8, num_aspects * 0.4 + 3)
        
        fig, ax = plt.subplots(figsize=(14, fig_height))
        
        # Vẽ biểu đồ từ DataFrame
        support_df.plot(
            kind='barh', 
            stacked=True, 
            color=colors, # Sử dụng bảng màu đã lọc
            ax=ax,
            width=0.8
        )
        
        ax.set_title(f"Phân bố Cảm xúc (Không tính 'None') - {title}", fontsize=18, pad=20)
        ax.set_xlabel('Số lượng mẫu (Support)', fontsize=14)
        ax.set_ylabel('Khía cạnh (Aspect)', fontsize=14)
        
        # Di chuyển legend ra ngoài biểu đồ
        ax.legend(
            title='Cảm xúc (Sentiment)',
            bbox_to_anchor=(1.02, 1), 
            loc='upper left', 
            fontsize='medium'
        )
        
        # Hiển thị lưới
        ax.grid(axis='x', linestyle='--', alpha=0.7)
        ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
        
        plt.margins(y=0.01)
        
        full_output_path = os.path.join(output_dir, output_filename)
        
        # Điều chỉnh layout để chừa chỗ cho legend
        fig.tight_layout(rect=[0, 0, 0.85, 1]) 
        
        fig.savefig(full_output_path)
        print(f"Đã lưu biểu đồ thành công vào: {full_output_path}")
        plt.close(fig) # Đóng Figure

    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý {csv_path}: {e}")

# --- Chạy thực thi ---
print("Bắt đầu trực quan hóa support (chỉ các nhãn 1, 2, 3)...")

# 1. Vẽ biểu đồ cho bộ Hotel
plot_active_support_per_aspect(
    csv_path=hotel_train_path,
    title="UIT-Hotel-ABSA (Train Set)",
    output_filename="hotel_active_support_per_aspect.png",
    label_map=label_map_no_none,
    colors=colors_no_none
)

# 2. Vẽ biểu đồ cho bộ Restaurant
plot_active_support_per_aspect(
    csv_path=restaurant_train_path,
    title="UIT-Restaurant-ABSA (Train Set)",
    output_filename="restaurant_active_support_per_aspect.png",
    label_map=label_map_no_none,
    colors=colors_no_none
)

print("\nHoàn tất quá trình.")

Sẽ lưu biểu đồ vào thư mục: /kaggle/working/visualizations_active_sentiments
Bắt đầu trực quan hóa support (chỉ các nhãn 1, 2, 3)...

Đang xử lý: UIT-Hotel-ABSA (Train Set)
Kiểm tra tệp tại: /kaggle/input/uit-hotel-absa/Train.csv
Tìm thấy 34 cột khía cạnh.
Đã lưu biểu đồ thành công vào: visualizations_active_sentiments/hotel_active_support_per_aspect.png

Đang xử lý: UIT-Restaurant-ABSA (Train Set)
Kiểm tra tệp tại: /kaggle/input/uit-absa-resturant-dataset/Train.csv
Tìm thấy 12 cột khía cạnh.
Đã lưu biểu đồ thành công vào: visualizations_active_sentiments/restaurant_active_support_per_aspect.png

Hoàn tất quá trình.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# --- Thiết lập ---

# Đường dẫn tệp từ notebook của bạn
hotel_train_path = "/kaggle/input/uit-hotel-absa/Train.csv"

# Chỉ bao gồm các nhãn 1, 2, 3
label_map_no_none = {
    1: 'Positive (1)',
    2: 'Negative (2)',
    3: 'Neutral (3)'
}
# Màu sắc tương ứng
colors_no_none = ['#4CAF50', '#F44336', '#FFC107'] 

# Thư mục để lưu hình ảnh
output_dir = "visualizations_active_sentiments"
os.makedirs(output_dir, exist_ok=True)
print(f"Sẽ lưu biểu đồ vào thư mục: {os.path.abspath(output_dir)}")

def process_hotel_data(csv_path, label_map):
    """
    Tải và xử lý dữ liệu Hotel, chỉ trả về support cho các nhãn 1, 2, 3.
    """
    if not os.path.exists(csv_path):
        print(f"--- CẢNH BÁO: Không tìm thấy tệp tại đường dẫn: {csv_path} ---")
        return None
    
    try:
        df = pd.read_csv(csv_path)
        aspect_cols = [col for col in df.columns if col != 'Review']
        print(f"Tìm thấy {len(aspect_cols)} cột khía cạnh.")
        
        # Đếm support
        support_df = df[aspect_cols].apply(pd.Series.value_counts).fillna(0).astype(int)
        
        # Đảm bảo các cột (0, 1, 2, 3) đều tồn tại
        support_df = support_df.reindex([0, 1, 2, 3], fill_value=0)
        
        # --- Loại bỏ hàng 'None (0)' ---
        if 0 in support_df.index:
            support_df = support_df.drop(0)
        
        # Chuyển vị và đổi tên cột
        support_df = support_df.transpose()
        support_df = support_df.rename(columns=label_map)
        
        # --- Sắp xếp theo thứ tự bảng chữ cái (A-Z) ---
        # ascending=False để khi vẽ barh, 'A' ở trên cùng
        support_df = support_df.sort_index(ascending=False) 
        
        return support_df

    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý {csv_path}: {e}")
        return None

def plot_subplot(df, ax, title_suffix):
    """
    Hàm trợ giúp để vẽ một biểu đồ con (subplot).
    """
    df.plot(
        kind='barh', 
        stacked=True, 
        color=colors_no_none,
        ax=ax,
        width=0.8
    )
    
    ax.set_title(f"Phân bố Cảm xúc Hotel (Phần {title_suffix})", fontsize=16, pad=15)
    ax.set_xlabel('Số lượng mẫu (Support)', fontsize=12)
    ax.set_ylabel('Khía cạnh (Aspect)', fontsize=12)
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

# --- Chạy thực thi ---
print("Bắt đầu trực quan hóa support Hotel (chỉ các nhãn 1, 2, 3) với 2 subplots...")

# 1. Xử lý dữ liệu
hotel_support_df = process_hotel_data(hotel_train_path, label_map_no_none)

if hotel_support_df is not None:
    # 2. Chia DataFrame đã sắp xếp thành 2 phần
    df_parts = np.array_split(hotel_support_df, 2)
    df_part1 = df_parts[0]
    df_part2 = df_parts[1]
    
    print(f"Đã chia 34 khía cạnh thành 2 nhóm: Nhóm 1 ({len(df_part1)} khía cạnh), Nhóm 2 ({len(df_part2)} khía cạnh)")
    
    # 3. Tính toán chiều cao
    # Chiều cao cho mỗi subplot
    fig_height_per_plot = max(8, len(df_part1) * 0.45 + 3)
    
    # 4. Tạo 2 subplots (2 hàng, 1 cột)
    fig, (ax1, ax2) = plt.subplots(
        2, 1, # 2 hàng, 1 cột
        figsize=(14, fig_height_per_plot * 2.1), # Nhân đôi chiều cao tổng
        sharex=True # Chia sẻ trục X
    )
    
    # 5. Vẽ từng subplot
    plot_subplot(df_part1, ax1, "1 / 2")
    plot_subplot(df_part2, ax2, "2 / 2")
    
    # 6. Tạo một legend chung cho cả Figure
    handles, labels = ax1.get_legend_handles_labels()
    fig.legend(
        handles, 
        labels, 
        title='Cảm xúc (Sentiment)',
        bbox_to_anchor=(1.0, 0.9), # Đặt legend bên ngoài
        loc='upper left', 
        fontsize='medium'
    )
    
    fig.suptitle("Phân bố Cảm xúc (Không tính 'None') - UIT-Hotel-ABSA (Train Set)", fontsize=20, y=1.03)
    
    # 7. Điều chỉnh layout và lưu
    full_output_path = os.path.join(output_dir, "hotel_active_support_subplots.png")
    fig.tight_layout(rect=[0, 0, 0.88, 1]) # Chừa không gian cho legend chung
    
    fig.savefig(full_output_path)
    print(f"Đã lưu biểu đồ thành công vào: {full_output_path}")
    plt.close(fig) # Đóng Figure
else:
    print("Không thể tạo biểu đồ do không tìm thấy hoặc lỗi xử lý tệp dữ liệu.")

print("\nHoàn tất quá trình.")

Sẽ lưu biểu đồ vào thư mục: /kaggle/working/visualizations_active_sentiments
Bắt đầu trực quan hóa support Hotel (chỉ các nhãn 1, 2, 3) với 2 subplots...
Tìm thấy 34 cột khía cạnh.
Đã chia 34 khía cạnh thành 2 nhóm: Nhóm 1 (17 khía cạnh), Nhóm 2 (17 khía cạnh)


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Đã lưu biểu đồ thành công vào: visualizations_active_sentiments/hotel_active_support_subplots.png

Hoàn tất quá trình.


In [5]:
import torch
import pandas as pd
from transformers import AutoTokenizer
import os

# --- Thiết lập ---
# Đảm bảo các đường dẫn này khớp với notebook của bạn
hotel_train_path = "/kaggle/input/uit-hotel-absa/Train.csv"
restaurant_train_path = "/kaggle/input/uit-absa-resturant-dataset/Train.csv"
tokenizer_name = "VietAI/vit5-base" # Hoặc 'VietAI/vit5-large' tùy bạn dùng

# Ánh xạ nhãn (Giống Slide 4 của bạn)
label_map_text = {
    0: 'None',
    1: 'Positive',
    2: 'Negative',
    3: 'Neutral'
}

def print_examples(dataset, num_examples=3, dataset_title=""):
    """
    In ra các ví dụ câu và nhãn active (khác None) từ dataset.
    """
    print("\n" + "="*50)
    print(f"    VÍ DỤ MẪU TỪ BỘ DỮ LIỆU: {dataset_title} ")
    print("="*50)
    
    if dataset is None:
        print("Không thể tải dataset (có thể do lỗi đường dẫn).")
        return

    aspect_names = dataset.aspect_columns
    
    for i in range(num_examples):
        # Lấy 3 mẫu đầu tiên
        try:
            example = dataset[i]
        except IndexError:
            print("Dataset không đủ mẫu để hiển thị.")
            break
            
        text = example['review_text']
        # labels là tensor [Num_Aspects, 4]
        labels_tensor = example['labels']
        
        # Chuyển one-hot -> index (0, 1, 2, hoặc 3)
        label_indices = torch.argmax(labels_tensor, dim=1) 
        
        print(f"\n--- VÍ DỤ {i+1} ---")
        print(f"Review: \"{text}\"")
        print("Labels (Active):")
        
        active_labels_found = False
        for j, aspect_name in enumerate(aspect_names):
            sentiment_index = label_indices[j].item()
            sentiment_text = label_map_text[sentiment_index]
            
            # Chỉ in nếu nhãn không phải là 'None'
            if sentiment_text != 'None':
                print(f"  - {{{aspect_name}, {sentiment_text}}}")
                active_labels_found = True
        
        if not active_labels_found:
            print("  (Không có khía cạnh nào được gán nhãn trong câu này)")
        
        print("-" * 25)

# --- Thực thi ---

# Giả định cell định nghĩa class ABSADataset đã được chạy

try:
    print("Đang tải tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    print(f"Đang tải dataset Hotel từ: {hotel_train_path}")
    hotel_dataset = None
    if os.path.exists(hotel_train_path):
        # Lớp ABSADataset phải được định nghĩa ở cell trước
        hotel_dataset = ABSADataset(hotel_train_path, tokenizer, max_length=128)
    else:
        print(f"CẢNH BÁO: Không tìm thấy tệp {hotel_train_path}")

    print(f"Đang tải dataset Restaurant từ: {restaurant_train_path}")
    restaurant_dataset = None
    if os.path.exists(restaurant_train_path):
        # Lớp ABSADataset phải được định nghĩa ở cell trước
        restaurant_dataset = ABSADataset(restaurant_train_path, tokenizer, max_length=128)
    else:
        print(f"CẢNH BÁO: Không tìm thấy tệp {restaurant_train_path}")
    
    # In ví dụ cho Hotel
    print_examples(hotel_dataset, num_examples=3, dataset_title="UIT-Hotel-ABSA")
    
    # In ví dụ cho Restaurant
    print_examples(restaurant_dataset, num_examples=3, dataset_title="UIT-Restaurant-ABSA")

except NameError as e:
    if 'ABSADataset' in str(e):
        print("\n--- LỖI ---")
        print("Bạn cần chạy cell định nghĩa class 'ABSADataset' trước khi chạy cell này.")
        print("-----------")
    else:
        raise e
except Exception as e:
    print(f"\nĐã xảy ra lỗi: {e}")
    print("Hãy đảm bảo rằng bạn đã cài đặt thư viện 'transformers' và đường dẫn tệp là chính xác.")

Đang tải tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Đang tải dataset Hotel từ: /kaggle/input/uit-hotel-absa/Train.csv

--- LỖI ---
Bạn cần chạy cell định nghĩa class 'ABSADataset' trước khi chạy cell này.
-----------


In [6]:
import pandas as pd
import os

# --- Thiết lập ---
# Đường dẫn tệp từ các notebook của bạn
hotel_train_path = "/kaggle/input/uit-hotel-absa/Train.csv"
restaurant_train_path = "/kaggle/input/uit-absa-resturant-dataset/Train.csv"

# Ánh xạ nhãn đơn giản (chỉ 1, 2, 3 theo yêu cầu của bạn)
label_map_text_gold = {
    1: 'positive',
    2: 'negative',
    3: 'neutral'
}

def print_simple_examples(csv_path, dataset_title, num_examples=5):
    """
    Hàm này CHỈ dùng Pandas để đọc N dòng đầu và in ví dụ.
    Không cần Torch, Transformers, hay bất kỳ class tùy chỉnh nào.
    """
    print("\n" + "="*50)
    print(f"    VÍ DỤ MẪU TỪ BỘ DỮ LIỆU: {dataset_title} ")
    print("="*50)
    
    # Kiểm tra xem tệp có tồn tại không
    if not os.path.exists(csv_path):
        print(f"--- CẢNH BÁO: Không tìm thấy tệp tại đường dẫn: {csv_path} ---")
        print("Hãy đảm bảo bạn đã thêm bộ dữ liệu vào notebook (mục 'Input').")
        print("--------------------------------------------------")
        return

    try:
        # Chỉ đọc N dòng đầu tiên
        df = pd.read_csv(csv_path, nrows=num_examples)
        
        # Xác định các cột khía cạnh (tất cả các cột TRỪ 'Review')
        aspect_cols = [col for col in df.columns if col != 'Review']
        
        # Lặp qua từng hàng (mỗi hàng là một review)
        for index, row in df.iterrows():
            review_text = row['Review']
            
            print(f"\n--- VÍ DỤ {index + 1} ---")
            print(f"Review: \"{review_text}\"")
            
            active_labels_list = []
            # Lặp qua từng cột khía cạnh cho hàng hiện tại
            for aspect_name in aspect_cols:
                sentiment_index = row[aspect_name] # Đây là số 0, 1, 2, hoặc 3
                
                # Chỉ lấy các nhãn 1, 2, hoặc 3
                if sentiment_index in label_map_text_gold:
                    sentiment_text = label_map_text_gold[sentiment_index]
                    label_string = f"{{{aspect_name}, {sentiment_text}}}"
                    active_labels_list.append(label_string)
            
            # In tất cả các nhãn active
            if active_labels_list:
                print(f"Labels: {', '.join(active_labels_list)}")
            else:
                print(f"Labels: (Không có khía cạnh nào được gán nhãn 1, 2, hoặc 3)")
            
            print("-" * 25)

    except Exception as e:
        print(f"Đã xảy ra lỗi khi đọc {csv_path}: {e}")

# --- Chạy thực thi ---
print_simple_examples(hotel_train_path, "VLSP2018-Hotel-ABSA", num_examples=5)
print_simple_examples(restaurant_train_path, "VLSP2018-Restaurant-ABSA", num_examples=5)


    VÍ DỤ MẪU TỪ BỘ DỮ LIỆU: VLSP2018-Hotel-ABSA 

--- VÍ DỤ 1 ---
Review: "Vừa qua tôi có dùng dịch vụ tại Khách Sạn TTC Hotel Premium Ngọc Lan Ngọc Lan Đà Lạt."
Labels: {HOTEL#GENERAL, positive}, {SERVICE#GENERAL, positive}
-------------------------

--- VÍ DỤ 2 ---
Review: "Tuy nhiên buffet sáng ở đây không được ngon và chưa đa dạng lắm."
Labels: {FOOD&DRINKS#QUALITY, negative}, {FOOD&DRINKS#STYLE&OPTIONS, negative}
-------------------------

--- VÍ DỤ 3 ---
Review: "Nhìn chung dịch vụ khách sạn cũng tốt, chỉ có đồ ăn sáng chưa ngon, chưa hợp khẩu vị với tôi."
Labels: {FOOD&DRINKS#QUALITY, negative}, {SERVICE#GENERAL, positive}
-------------------------

--- VÍ DỤ 4 ---
Review: "Đội ngũ nhân viên phục vụ chu đáo, tận tình."
Labels: {SERVICE#GENERAL, positive}
-------------------------

--- VÍ DỤ 5 ---
Review: "Trang thiết bị nội thất hài hoà, tiện nghi hiện đại, phù hợp với tiêu chuẩn khách sạn."
Labels: {HOTEL#QUALITY, positive}, {ROOM_AMENITIES#DESIGN&FEATURES, positive}, {ROOM_A